In [1]:
import httpx
import json
from typing import Dict
from urllib.parse import quote


In [2]:
def scrape_post(ig_doc_id ,url_or_shortcode: str) -> Dict:
    """Scrape single Instagram post data"""
    print(f"Scraping: {url_or_shortcode}, id: {ig_doc_id}")
    
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode

    variables = quote(json.dumps({
        'shortcode':shortcode,'fetch_tagged_user_count':None,
        'hoisted_comment_id':None,'hoisted_reply_id':None
    }, separators=(',', ':')))
    body = f"variables={variables}&doc_id={ig_doc_id}"
    url = "https://www.instagram.com/graphql/query"

    result = httpx.post(
        url=url,
        headers={"content-type": "application/x-www-form-urlencoded"},
        data=body
    )
    data = json.loads(result.content)
    
    print(data)
    
    # return data["data"]["xdt_shortcode_media"]
    return data

In [ ]:
# document id for content query:
#more post content: 
# IG_DOC_ID = "9195084877213487"
#more content
# IG_DOC_ID = "9507655305965624"



#relationships/related : 9480525715305126
# IG_DOC_ID = "9480525715305126"
#acc metadata: 9109150515847101
# IG_DOC_ID = "9109150515847101"
#reels: 9318096484942317
# IG_DOC_ID = "9318096484942317"

In [3]:
 ! pip install pytest-playwright
 ! playwright install

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import re
import json

from dotenv import load_dotenv
#load the .env file
# get USERNAME and PASSWORD
load_dotenv(override=True)
import os
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [ ]:
#signon doesn't work rn

import asyncio
from playwright.async_api import TimeoutError

async def signon(page, username, password):
    try:
        await page.goto("https://www.instagram.com/accounts/login/", wait_until="networkidle")

        await page.wait_for_selector('input[name="username"]', timeout=10000)
        print("Filling in login form...")
        await page.fill('input[name="username"]', username)
        await page.fill('input[name="password"]', password)
        #sleep 500ms
        await page.wait_for_timeout(500)
        print("Hitting login button")
        
        # Wait for the login button to be visible (both selectors)
        # await page.getByRole('button', { aria-label=="Log in" }).click();
        await page.locator('[role="button"][aria-label-id="replay"]').click();


        # wait for networkidle
        await page.wait_for_timeout(5000)
        
        
        # print("Networkidle #1")
        # wait for things to settle
        await page.wait_for_timeout(1000)
        # Check for any popups or modals

        # Verify successful login by checking URL or user-specific elements
        if "login" in page.url:
            raise ValueError("Login failed: Incorrect username or password.")

        print("✅ Successfully logged in")
        
        return

    except TimeoutError:
        raise TimeoutError("Login timed out. Check credentials or network connection.")
    
    

# Initiate & Sign in

In [61]:
#This is a dynamic environment that stays open after execution

# Global references for easy access across notebook cells
playwright = await async_playwright().start()
device = playwright.devices["Pixel 5"]
browser = await playwright.chromium.launch(headless=False)  # Browser will stay open
context = await browser.new_context(**device)
page = await context.new_page()
# You now have persistent global references:
# playwright, browser, context, page
#it says it fails but it works
await signon(page, username, password)

Filling in login form...
Hitting login button


TimeoutError: Login timed out. Check credentials or network connection.

Future exception was never retrieved
future: <Future finished exception=Exception('Connection closed while reading from the driver')>
Exception: Connection closed while reading from the driver
Future exception was never retrieved
future: <Future finished exception=Exception('Connection closed while reading from the driver')>
Exception: Connection closed while reading from the driver
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("button[tabindex=\\"0\\"]")\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("button[tabindex=\"0\"]")

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("[role=\\"button\\"][aria-label-id=\\"replay\\"]")\n  -     - waiting for" htt

✅ Intercepted post 3604316670938575496_54284618 (1 collected)
✅ Intercepted post 3597918104681722762_54284618 (2 collected)
✅ Intercepted post 3597918104681722762_54284618 (2 collected)
✅ Intercepted post 3590625013067163752_54284618 (3 collected)
✅ Intercepted post 3590625013067163752_54284618 (3 collected)
✅ Intercepted post 3590625013067163752_54284618 (3 collected)
✅ Intercepted post 3595683159321203024_54284618 (4 collected)
✅ Intercepted post 3595683159321203024_54284618 (4 collected)
✅ Intercepted post 3595683159321203024_54284618 (4 collected)
✅ Intercepted post 3595683159321203024_54284618 (4 collected)
✅ Intercepted post 3572307178285191786_54284618 (5 collected)
✅ Intercepted post 3572307178285191786_54284618 (5 collected)
✅ Intercepted post 3572307178285191786_54284618 (5 collected)
✅ Intercepted post 3572307178285191786_54284618 (5 collected)
✅ Intercepted post 3572307178285191786_54284618 (5 collected)
✅ Intercepted post 3585524311525585132_54284618 (6 collected)
✅ Interc

In [49]:
try:
    # await page.wait_for_selector('button[aria-label="Close"]', timeout=5000)
    # await page.click('button[aria-label="Close"]')
    # close button is tabindex=0
    await page.click('button[tabindex="0"]')
except TimeoutError:
    pass  # No popup appeared, continue

CancelledError: 

In [17]:
from random import randint

In [ ]:
import asyncio
import re
import json
from random import randint
from datetime import datetime

MAX_SCROLLS = 10
SCROLL_DELAY = 3  # Time between scrolls in seconds
testname = "mattrife"
profile_url = f"https://www.instagram.com/{testname}/"

### ========================= UTILITY FUNCTIONS ========================= ###
async def get_total_posts(page):
    """Extracts the total number of posts from the user's profile."""
    total_posts = await page.evaluate("""
        () => {
            let element = document.querySelector('header section ul li span');
            return element ? parseInt(element.innerText.replace(',', '')) : null;
        }
    """)
    return total_posts if total_posts else float('inf')

async def get_post_date(post_element):
    """Extracts the post date from a post element."""
    timestamp = await post_element.evaluate("""
        node => {
            let timeElem = node.querySelector('time');
            return timeElem ? timeElem.getAttribute('datetime') : null;
        }
    """)
    return datetime.fromisoformat(timestamp) if timestamp else None

async def intercept_response(response, intercepted_posts, response_received_flag):
    """Intercepts post data from Instagram API responses."""
    endpoint_pattern = re.compile(r"/api/v1/media/\d+/info/")
    if endpoint_pattern.search(response.url) and response.status == 200:
        data = await response.json()
        media_id = data.get('items', [{}])[0].get('id', 'unknown')
        intercepted_posts[media_id] = data
        response_received_flag[0] = True
        print(f"✅ Intercepted post {media_id} ({len(intercepted_posts)} collected)")

### ========================= SCROLLING FUNCTION ========================= ###
async def scroll_to_load_all_posts(page, max_scrolls=MAX_SCROLLS):
    all_posts = set()
    previous_count = 0
    scroll_attempts = 0

    print(f"🔄 Starting scrolling process... (Max {max_scrolls} attempts)")

    while scroll_attempts < max_scrolls:
        posts = await page.query_selector_all("div._aagu")
        if not posts:
            print("⚠️ No posts found in current DOM snapshot.")
            break
        
        for post in posts:
            parent = await post.evaluate_handle("node => node.closest('a')")
            if parent:
                href = await parent.get_attribute('href')
                if href and '/p/' in href:
                    if href not in all_posts:  # Only process new posts
                        all_posts.add(href)

        if len(all_posts) == previous_count:
            print(f"✅ Scrolling stopped after {scroll_attempts} attempts. Total unique posts found: {len(all_posts)}")
            break  # Stop if no new posts are loaded

        previous_count = len(all_posts)
        scroll_attempts += 1

        # Scroll and use timer-based waiting
        print(f"📜 Scrolling attempt {scroll_attempts}... (Loaded posts: {len(all_posts)})")
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")

        # Use a fixed timer to wait for new posts
        await asyncio.sleep(SCROLL_DELAY)

    print(f"✅ Finished scrolling. Total posts loaded: {len(all_posts)}")
    return all_posts

### ========================= NAVIGATION & DATA COLLECTION ========================= ###
async def navigate_and_collect(page, all_post_links):
    """Visits each post URL and collects data via API interception."""
    intercepted_posts = {}
    
    print(f"🟢 Total unique posts queued for processing: {len(all_post_links)}")

    for i, href in enumerate(all_post_links, start=1):
        post_url = f"https://www.instagram.com{href}"
        print(f"\n({i}/{len(all_post_links)}) 📌 Navigating to: {post_url}")

        await page.goto(post_url, wait_until="networkidle")
        await asyncio.sleep(1)  # Allow responses to load

        # Ensure at least one response is intercepted
        response_received_flag = [False]
        page.on("response", lambda response: intercept_response(response, intercepted_posts, response_received_flag))

        # Wait dynamically for response interception (max 5 seconds)
        for _ in range(50):
            if response_received_flag[0]:
                break
            await asyncio.sleep(0.1)

        # Return to profile
        print("🔄 Returning to profile page...\n")
        await page.goto("about:blank")  # Clear previous page before navigating back
        await page.goto(profile_url, wait_until="networkidle")
        await asyncio.sleep(1)  # Shorter delay to speed things up

    print(f"\n🎉 **Scraping complete! Collected {len(intercepted_posts)} posts.**")
    return intercepted_posts

### ========================= MAIN SCRAPING FUNCTION ========================= ###
async def scrape_profile_posts(profile_url: str, page):
    """Orchestrates the entire scraping process."""
    print(f"🚀 Navigating to profile: {profile_url}")
    await page.goto(profile_url, wait_until="networkidle")
    await asyncio.sleep(1)

    total_posts = await get_total_posts(page)
    print(f"🔢 Total posts found on profile: {total_posts}")

    all_post_links = await scroll_to_load_all_posts(page)
    print(f"📸 Found {len(all_post_links)} unique posts after scrolling.")

    intercepted_posts = await navigate_and_collect(page, all_post_links)

    return intercepted_posts

### ========================= RUN SCRAPER ========================= ###
# Example usage:
data = await scrape_profile_posts("https://www.instagram.com/mattrife/", page)
print(json.dumps(data, indent=2))


🚀 Navigating to profile: https://www.instagram.com/mattrife/
🔢 Total posts found on profile: 1553
🔄 Starting scrolling process... (Max 10 attempts)
📜 Scrolling attempt 1... (Loaded posts: 8)
📜 Scrolling attempt 2... (Loaded posts: 13)
✅ Scrolling stopped after 2 attempts. Total unique posts found: 13
✅ Finished scrolling. Total posts loaded: 13
📸 Found 13 unique posts after scrolling.
🟢 Total unique posts queued for processing: 13

(1/13) 📌 Navigating to: https://www.instagram.com/mattrife/p/DGBYWJQRCNb/
🔄 Returning to profile page...


(2/13) 📌 Navigating to: https://www.instagram.com/mattrife/p/DIFGuptT_KI/
🔄 Returning to profile page...


(3/13) 📌 Navigating to: https://www.instagram.com/mattrife/p/DHuX3QggDOK/
🔄 Returning to profile page...


(4/13) 📌 Navigating to: https://www.instagram.com/mattrife/p/DHUdmzpgCho/
🔄 Returning to profile page...


(5/13) 📌 Navigating to: https://www.instagram.com/mattrife/p/DHmbshcMdFQ/
🔄 Returning to profile page...


(6/13) 📌 Navigating to: https

In [38]:
len(data)
# data÷

9

In [31]:
# save the data to a json file
with open('mattrife_posts.json', 'w') as f:
    json.dump(data, f)